In [91]:

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#NLTK-------------------------------
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.snowball import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
#from nltk.stemporter import PorterStemmer

# Import libraries for feature 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.model_selection import RandomizedSearchCV


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")


from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [60]:
#Read files
textfile = r'/gdrive/My Drive/CIS508A5/Comments.csv'
textData = pd.read_csv(textfile) #creates a dataframe

CustInfofile = r'/gdrive/My Drive/CIS508A5/Customers.csv'
CustInfoData = pd.read_csv(CustInfofile)  #creates a dataframe

print(textData.shape)
print(CustInfoData.shape)


(2070, 2)
(2070, 17)


In [61]:
#Extract target column from Customer Info file
y_train = CustInfoData["TARGET"]
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
                     
print(X_train.shape)
print(textData.shape)
textData.head()
print(y_train)

(2070, 16)
(2070, 2)
0       Cancelled
1         Current
2         Current
3         Current
4       Cancelled
          ...    
2065    Cancelled
2066    Cancelled
2067    Cancelled
2068    Cancelled
2069    Cancelled
Name: TARGET, Length: 2070, dtype: object


In [0]:
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)  #Tokenization means breaking down words eg. audit becomes adit 


In [0]:
snowball = SnowballStemmer("english")
#Now do stemming - create a new dataframe to store stemmed version
snowballTextData=pd.DataFrame()
snowballTextData=textData.drop(columns=["CommentsTokenized","Comments"])
snowballTextData['CommentsTokenizedSnowball'] = textData['CommentsTokenized'].apply(lambda x: [snowball.stem(y) for y in x]) # Stem every word.

#export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/newTextDataTS.csv')

#Join stemmed strings
snowballTextData['CommentsTokenizedSnowball'] = snowballTextData['CommentsTokenizedSnowball'].apply(lambda x: " ".join(x))

export_csv = snowballTextData.to_csv(r'/gdrive/My Drive/CIS508A5/snowballTextData-Joined.csv')

In [0]:
porter = PorterStemmer()
#Now do stemming - create a new dataframe to store stemmed version
porterTextData=pd.DataFrame()
porterTextData=textData.drop(columns=["CommentsTokenized","Comments"])
porterTextData['CommentsTokenizedPorter'] = textData['CommentsTokenized'].apply(lambda x: [porter.stem(y) for y in x]) # Stem every word.

#export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/newTextDataTS.csv')

#Join stemmed strings
porterTextData['CommentsTokenizedPorter'] = porterTextData['CommentsTokenizedPorter'].apply(lambda x: " ".join(x))

export_csv = porterTextData.to_csv(r'/gdrive/My Drive/CIS508A5/porterTextData-Joined.csv')

In [0]:
lanc = LancasterStemmer()
#Now do stemming - create a new dataframe to store stemmed version
lancTextData=pd.DataFrame()
lancTextData=textData.drop(columns=["CommentsTokenized","Comments"])
lancTextData['CommentsTokenizedlanc'] = textData['CommentsTokenized'].apply(lambda x: [lanc.stem(y) for y in x]) # Stem every word.

#export_csv = newTextData.to_csv(r'/gdrive/My Drive/CIS508/newTextDataTS.csv')

#Join stemmed strings
lancTextData['CommentsTokenizedlanc'] = lancTextData['CommentsTokenizedlanc'].apply(lambda x: " ".join(x))

export_csv = lancTextData.to_csv(r'/gdrive/My Drive/CIS508A5/lancTextData-Joined.csv')

In [66]:
#Do Bag-Of-Words model - Term - Document Matrix
#Learn the vocabulary dictionary and return term-document matrix.
#count_vect = CountVectorizer(stop_words=None)
count_vect = CountVectorizer(stop_words='english',lowercase=False)

TD_counts_porter = count_vect.fit_transform(porterTextData.CommentsTokenizedPorter)
TD_counts_lanc = count_vect.fit_transform(lancTextData.CommentsTokenizedlanc)
TD_counts_snowball = count_vect.fit_transform(snowballTextData.CommentsTokenizedSnowball)


# Count number of stemmed words from each of the Stemmers
print('Number of stemmed words by Porter:',TD_counts_porter.shape[1])
print('Number of stemmed words by Lancaster:',TD_counts_lanc.shape[1])
print('Number of stemmed words by Snowball:',TD_counts_snowball.shape[1])



Number of stemmed words by Porter: 366
Number of stemmed words by Lancaster: 364
Number of stemmed words by Snowball: 354


In [67]:
TD_counts_porter.dtype
print(count_vect.get_feature_names())
print(TD_counts_porter)
DF_TD_Counts=pd.DataFrame(TD_counts_porter.toarray())
print(DF_TD_Counts)


['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effect', 'encount', 'e

In [68]:
TD_counts_lanc.dtype
print(count_vect.get_feature_names())
print(TD_counts_lanc)
DF_TD_Counts=pd.DataFrame(TD_counts_lanc.toarray())
print(DF_TD_Counts)


['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effect', 'encount', 'e

In [69]:
TD_counts_snowball.dtype
print(count_vect.get_feature_names())
print(TD_counts_snowball)
DF_TD_Counts=pd.DataFrame(TD_counts_snowball.toarray())
print(DF_TD_Counts)


['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effect', 'encount', 'e

In [70]:
#Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(TD_counts_porter)
print(X_train_tfidf.shape)
DF_TF_IDF_porter=pd.DataFrame(X_train_tfidf.toarray())
print(DF_TF_IDF_porter)


(2070, 366)
      0    1    2    3         4    5    ...  360  361  362  363  364  365
0     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
1     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
3     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
4     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
...   ...  ...  ...  ...       ...  ...  ...  ...  ...  ...  ...  ...  ...
2065  0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2066  0.0  0.0  0.0  0.0  0.407251  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2067  0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2068  0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2069  0.0  0.0  0.0  0.0  0.407251  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0

[2070 rows x 366 columns]


In [71]:
#Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(TD_counts_lanc)
print(X_train_tfidf.shape)
DF_TF_IDF_lanc=pd.DataFrame(X_train_tfidf.toarray())
print(DF_TF_IDF_lanc)


(2070, 364)
      0    1    2         3    4    5    6    ...  357  358  359  360  361  362  363
0     0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1     0.0  0.0  0.0  0.275669  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2     0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3     0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4     0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
...   ...  ...  ...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
2065  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2066  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2067  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2068  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2069  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.

In [0]:
#Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(TD_counts_snowball)
print(X_train_tfidf.shape)
DF_TF_IDF_snowball=pd.DataFrame(X_train_tfidf.toarray())
print(DF_TF_IDF_snowball)


In [0]:
#Feature selection Filter type
new_DF_TF_IDF = SelectKBest(score_func=chi2, k=150).fit_transform(DF_TF_IDF_porter,y_train)
new_DF_TF_IDF.shape

DF_TF_IDF_Filter= pd.DataFrame(new_DF_TF_IDF)
print(DF_TF_IDF_Filter)


In [0]:
#Construct a Random Forest Classifier on the filter text data
clf=RandomForestClassifier()
RF_text = clf.fit(DF_TF_IDF_Filter,y_train)
print("Accuracy score (training): {0:.6f}".format(clf.score(DF_TF_IDF_Filter, y_train)))
rf_predictions = clf.predict(DF_TF_IDF_Filter)
print("Confusion Matrix:")
print(confusion_matrix(y_train, rf_predictions))
print("Classification Report")
print(classification_report(y_train, rf_predictions))

In [110]:
#Feature selection Wrapper Type

# Build RF classifier to use in feature selection
clf = RandomForestClassifier()

# Build step forward feature selection
sfs1 = sfs(clf,
           k_features=15,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)
sfs1 = sfs1.fit(DF_TF_IDF, y_train)
feat_cols = list(sfs1.k_feature_idx_)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 354 out of 354 | elapsed:   26.9s finished

[2019-12-13 17:03:57] Features: 1/15 -- score: 0.6159469643320449[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 353 out of 353 | elapsed:   27.1s finished

[2019-12-13 17:04:24] Features: 2/15 -- score: 0.6202889643988454[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 352 out of 352 | elapsed:   27.7s finished

[2019-12-13 17:04:52] Features: 3/15 -- score: 0.6241525402589408[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [0]:
DF_TF_IDF_Wrap=DF_TF_IDF.iloc[:,feat_cols]

In [112]:
#Construct a Random Forest Classifier on the wrap text data
clf=RandomForestClassifier()
RF_text = clf.fit(DF_TF_IDF_Wrap,y_train)
print("Accuracy score (training): {0:.6f}".format(clf.score(DF_TF_IDF_Wrap, y_train)))
rf_predictions = clf.predict(DF_TF_IDF_Wrap)
print("Confusion Matrix:")
print(confusion_matrix(y_train, rf_predictions))
print("Classification Report")
print(classification_report(y_train, rf_predictions))

Accuracy score (training): 0.632367
Confusion Matrix:
[[  65  739]
 [  22 1244]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.75      0.08      0.15       804
     Current       0.63      0.98      0.77      1266

    accuracy                           0.63      2070
   macro avg       0.69      0.53      0.46      2070
weighted avg       0.67      0.63      0.53      2070



In [78]:
#Merge files

print(CustInfoData.shape)
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
print(X_train.shape)
combined=pd.concat([X_train, DF_TF_IDF_SelectedFeatures], axis=1)
print(combined.shape)
print(combined)
export_csv= combined.to_csv(r'/gdrive/My Drive/CIS508A5/Combined-Cust+TFIDF+SelectedFeatures.csv')



(2070, 17)
(2070, 16)
(2070, 26)
        ID Sex Status  Children  Est_Income  ...    5    6    7    8    9
0        1   F      S         1    38000.00  ...  0.0  0.0  0.0  0.0  0.0
1        6   M      M         2    29616.00  ...  0.0  0.0  0.0  0.0  0.0
2        8   M      M         0    19732.80  ...  0.0  0.0  0.0  0.0  0.0
3       11   M      S         2       96.33  ...  0.0  0.0  0.0  0.0  0.0
4       14   F      M         2    52004.80  ...  0.0  0.0  0.0  0.0  0.0
...    ...  ..    ...       ...         ...  ...  ...  ...  ...  ...  ...
2065  3821   F      S         0    78851.30  ...  0.0  0.0  0.0  0.0  0.0
2066  3822   F      S         1    17540.70  ...  0.0  0.0  0.0  0.0  0.0
2067  3823   F      M         0    83891.90  ...  0.0  0.0  0.0  0.0  0.0
2068  3824   F      M         2    28220.80  ...  0.0  0.0  0.0  0.0  0.0
2069  3825   F      S         0    28589.10  ...  0.0  0.0  0.0  0.0  0.0

[2070 rows x 26 columns]


In [0]:
#Do one Hot encoding for categorical features
X_cat = ["Sex","Status","Car_Owner","Paymethod","LocalBilltype","LongDistanceBilltype"]
#X_cat = combined.select_dtypes(exclude=['int','float64'])
print(X_cat)
combined_one_hot = pd.get_dummies(combined,columns=X_cat)
print(combined_one_hot.shape)
export_csv= combined_one_hot.to_csv(r'/gdrive/My Drive/CIS508A5/combined_one_hot.csv')



In [0]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(combined_one_hot, y_train, test_size=0.2, random_state=42)

In [83]:
#Construct a Random Forest Classifier on combined data
#clf1=RandomForestClassifier()
RF_Comb = clf.fit(X_Train,Y_Train)
print("Accuracy score (training): {0:.6f}".format(clf.score(X_Train,Y_Train)))
rf_predictions = clf.predict(X_Train)
print("Confusion Matrix:")
print(confusion_matrix(Y_Train, rf_predictions))
print("Classification Report")
print(classification_report(Y_Train, rf_predictions))

Accuracy score (training): 0.986111
Confusion Matrix:
[[640   7]
 [ 16 993]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.98      0.99      0.98       647
     Current       0.99      0.98      0.99      1009

    accuracy                           0.99      1656
   macro avg       0.98      0.99      0.99      1656
weighted avg       0.99      0.99      0.99      1656



In [86]:
#CONTRUCT A GRADIENT BOOSTING MODEL==============================
abc =GradientBoostingClassifier()
abc.fit(X_Train,Y_Train)
abc_predict=abc.predict(X_Test)
print('Final prediction score for ensemble methods: [%.8f]' % acc(Y_Test, abc_predict))
print("Confusion Matrix after STACKING for Boosting:")
print(confusion_matrix(Y_Test, abc_predict))
print("=== Classification Report ===")
print(classification_report(Y_Test, abc_predict))

Final prediction score for ensemble methods: [0.85748792]
Confusion Matrix after STACKING for Boosting:
[[120  37]
 [ 22 235]]
=== Classification Report ===
              precision    recall  f1-score   support

   Cancelled       0.85      0.76      0.80       157
     Current       0.86      0.91      0.89       257

    accuracy                           0.86       414
   macro avg       0.85      0.84      0.85       414
weighted avg       0.86      0.86      0.86       414



In [93]:
#Gradient Boosting Hyperparam Tuning ============================================================
search_grid={'n_estimators':[5,10,20,30,50],'learning_rate':[0.02,0.5]}
#Randomized Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_Train,Y_Train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

#Construct Gradient Boosting Trees using the best parameters
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_Train,Y_Train)
abc_predict = abc.predict(X_Test)
print('Final prediction score for ensemble methods: [%.8f]' % acc(Y_Test, abc_predict))
print("Confusion Matrix after STACKING for Boosting:")
print(confusion_matrix(Y_Test,abc_predict))
print("=== Classification Report ===")
print(classification_report(Y_Test,abc_predict))

{'n_estimators': 50, 'learning_rate': 0.5}
Final prediction score for ensemble methods: [0.86231884]
Confusion Matrix after STACKING for Boosting:
[[123  34]
 [ 23 234]]
=== Classification Report ===
              precision    recall  f1-score   support

   Cancelled       0.84      0.78      0.81       157
     Current       0.87      0.91      0.89       257

    accuracy                           0.86       414
   macro avg       0.86      0.85      0.85       414
weighted avg       0.86      0.86      0.86       414

